In [3]:
spark.stop()

In [4]:
import findspark
findspark.init('/opt/cloudera/parcels/SPARK2-2.4.0.cloudera2-1.cdh5.13.3.p0.1041012/lib/spark2/')
import os, findspark
os.environ['PYSPARK_PYTHON'] = '/home/tozeng/anaconda3/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/tozeng/anaconda3/bin/python'
import sys
sys.path.append('/home/eileen/nsf_data_ingestion/')
import zipfile
import zipimport
import io
import logging
logging.getLogger().setLevel(logging.INFO)
from pyspark.sql import functions
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import Window
from pyspark.sql.functions import rank, max, sum, desc
import zlib
import importlib
import os
from os import path
from shutil import copyfile
from shutil import rmtree
from subprocess import call
import pubmed_parser as pp
from nsf_data_ingestion.config import spark_config
from nsf_data_ingestion.objects import data_source_params


def create_session():
    logging.info('Creating Spark Session....')
    spark = SparkSession.builder.\
            appName('test_pubmed_parser_that').\
            config('spark.driver.memory', '60g').\
            config('spark.network.timeout', '600s').\
            config('spark.driver.maxResultSize', '10g').\
            config('spark.executor.memory', '30g').\
            config('spark.kryoserializer.buffer.max', '1g').\
            config('spark.cores.max', '40').\
            getOrCreate()
    spark.sparkContext.addPyFile('/home/eileen/nsf_data_ingestion/libraries/pubmed_parser-0.1.0-py3.6.egg')
    spark.sparkContext.addPyFile('/home/eileen/nsf_data_ingestion/libraries/Unidecode-1.1.1-py3.6.egg')
#         logging.info('Adding Libraries' + str(library))
#         spark.sparkContext.addPyFile(library)    # adding libraries
#         spark.sparkContext.addPyFile(library)

    logging.info("Session Created!")
    return spark



spark = create_session()
sc = spark.sparkContext

INFO:root:Creating Spark Session....
INFO:root:Session Created!


In [7]:
def parse_gzip_medline_str(gzip_str):
    import pubmed_parser as pp
    import unidecode
    
    filepath = gzip_str[0]
    xml_string = gzip_str[1]
    _, file_name = path.split(filepath)

    articles = pp.parse_medline_xml(xml_string)
    return [Row(file_name=file_name, **article_dict)
            for article_dict in articles]

In [8]:

docs = sc.wholeTextFiles('test/test/pubmed21n0001.xml.gz').repartition(1000)
dicts = docs.flatMap(parse_gzip_medline_str)
medline_df = dicts.toDF()

In [9]:
import pandas as pd
panda_df = pd.DataFrame(medline_df.head(5), columns=medline_df.columns)
panda_df

,abstract,affiliation,author,country,delete,file_name,issn_linking,journal,keywords,medline_ta,mesh_terms,nlm_unique_id,other_id,pmc,pmid,pubdate,title
0,,,AB Makar; KE McMartin; M Palese; TR Tephly,United States,False,pubmed21n0001.xml.gz,0006-2944,,,Biochem Med,D000445:Aldehyde Oxidoreductases; D000818:Anim...,0151424,,,1,1975,Formate assay in body fluids: application in m...
1,,,KS Bose; RH Sarma,United States,False,pubmed21n0001.xml.gz,0006-291X,,,Biochem Biophys Res Commun,D005583:Fourier Analysis; D009682:Magnetic Res...,0372516,,,2,1975,Delineation of the intimate details of the bac...
2,,,RJ Smith; RG Bryant,United States,False,pubmed21n0001.xml.gz,0006-291X,,,Biochem Biophys Res Commun,D000818:Animals; D001665:Binding Sites; D00210...,0372516,,,3,1975,Metal substitutions incarbonic anhydrase: a ha...
3,,,UN Wiesmann; S DiDonato; NN Herschkowitz,United States,False,pubmed21n0001.xml.gz,0006-291X,,,Biochem Biophys Res Commun,"D001692:Biological Transport; D002478:Cells, C...",0372516,,,4,1975,Effect of chloroquine on cultured fibroblasts:...
4,,,WA Hendrickson; KB Ward,United States,False,pubmed21n0001.xml.gz,0006-291X,,,Biochem Biophys Res Commun,D000818:Animals; D003063:Cnidaria; D003201:Com...,0372516,,,5,1975,Atomic models for the polypeptide backbones of...


In [ ]:
def parse_gzip_medline_str(gzip_str):
    import pubmed_parser as pp
    import unidecode
    
    filepath = gzip_str[0]
    xml_string = gzip_str[1]
    _, file_name = path.split(filepath)

    articles = pp.parse_medline_xml(bytes(xml_string, 'utf-8'))
    return [Row(file_name=file_name, **article_dict)
            for article_dict in articles]

In [ ]:

docs = sc.wholeTextFiles('test/test/pubmed21n0001.xml.gz').repartition(1000)
dicts = docs.flatMap(parse_gzip_medline_str)
medline_df = dicts.toDF()

In [4]:
import pandas as pd
panda_df_new = pd.DataFrame(medline_df.head(5), columns=medline_df.columns)
panda_df_new

,abstract,affiliations,authors,chemical_list,country,delete,doi,file_name,issn_linking,journal,...,medline_ta,mesh_terms,nlm_unique_id,other_id,pmc,pmid,pubdate,publication_types,references,title
0,,,AB Makar;KE McMartin;M Palese;TR Tephly,D005561:Formates; D002245:Carbon Dioxide; D000...,United States,False,10.1016/0006-2944(75)90147-7,pubmed21n0001.xml.gz,0006-2944,Biochemical medicine,...,Biochem Med,D000445:Aldehyde Oxidoreductases; D000818:Anim...,0151424,,,1,1975,D016428:Journal Article; D013487:Research Supp...,,Formate assay in body fluids: application in m...
1,,,KS Bose;RH Sarma,D009243:NAD; D009249:NADP,United States,False,10.1016/0006-291x(75)90482-9,pubmed21n0001.xml.gz,0006-291X,Biochemical and biophysical research communica...,...,Biochem Biophys Res Commun,D005583:Fourier Analysis; D009682:Magnetic Res...,0372516,,,2,1975,D016428:Journal Article; D013486:Research Supp...,,Delineation of the intimate details of the bac...
2,,,RJ Smith;RG Bryant,D002104:Cadmium; D002256:Carbonic Anhydrases; ...,United States,False,10.1016/0006-291x(75)90498-2,pubmed21n0001.xml.gz,0006-291X,Biochemical and biophysical research communica...,...,Biochem Biophys Res Commun,D000818:Animals; D001665:Binding Sites; D00210...,0372516,,,3,1975,D016428:Journal Article; D013487:Research Supp...,,Metal substitutions incarbonic anhydrase: a ha...
3,,,UN Wiesmann;S DiDonato;NN Herschkowitz,D003911:Dextrans; D002738:Chloroquine; D013429...,United States,False,10.1016/0006-291x(75)90506-9,pubmed21n0001.xml.gz,0006-291X,Biochemical and biophysical research communica...,...,Biochem Biophys Res Commun,"D001692:Biological Transport; D002478:Cells, C...",0372516,,,4,1975,D016428:Journal Article,,Effect of chloroquine on cultured fibroblasts:...
4,,,WA Hendrickson;KB Ward,D006422:Hemerythrin; D008667:Metalloproteins; ...,United States,False,10.1016/0006-291x(75)90508-2,pubmed21n0001.xml.gz,0006-291X,Biochemical and biophysical research communica...,...,Biochem Biophys Res Commun,D000818:Animals; D003063:Cnidaria; D003201:Com...,0372516,,,5,1975,D016428:Journal Article,,Atomic models for the polypeptide backbones of...


In [10]:
panda_df.columns

Index(['abstract', 'affiliation', 'author', 'country', 'delete', 'file_name',
       'issn_linking', 'journal', 'keywords', 'medline_ta', 'mesh_terms',
       'nlm_unique_id', 'other_id', 'pmc', 'pmid', 'pubdate', 'title'],
      dtype='object')

In [ ]:
!pwd

In [ ]:
import pubmed_parser as pp
path_f = "/home/eileen/medline_data/pubmed21n0974.xml.gz"
dict = pp.parse_medline_xml(path_f)
local_df = pd.DataFrame(dict)
local_df

In [ ]:
pp.__version__

In [ ]:
!pip list

In [ ]:
pp.__file__

In [ ]:
!ls /home/eileen/.local/lib/python3.7/site-packages/

In [ ]:
rdd0 = sc.parallelize(list(range(10000))).repartition(1000)

In [ ]:
def get_hostnode(e):
    import sys
    import socket
    
    return str(sys.version_info), str(socket.gethostname())


In [ ]:
def check_version(e):
    import pyarrow as pa
    hdfs = pa.hdfs.connect()
    
    return str(pa.__version__), str(hdfs)

In [ ]:
rdd0.map(check_version).distinct().collect()

In [ ]:
hdfs = pa.hdfs.connect()

In [ ]:
print(hdfs)

In [ ]:
rdd0.map(get_hostnode).distinct().collect()

In [ ]:
get_hostnode(2)

In [ ]:
!/home/tozeng/anaconda3/bin/python --version

In [ ]:
re.__version__

In [ ]:
test_p = spark.sparkContext.parallelize(file_paths)
test_p.collect()

In [ ]:
def find_journal(gzip):
    import re
    l = re.findall('<Title>[\s\S]*?<\/Title>', gzip[1])

    return(l)

In [ ]:
def test_medline_parquet(gzip):
    import pubmed_parser as pa
    dicts = pa.parse_medline_xml(gzip[1])
    return [Row(**doc) for doc in dicts]

In [ ]:
docs = spark.sparkContext.wholeTextFiles('test/test/pubmed21n0001.xml.gz')
dicts = docs.flatMap(find_journal)
dicts.take(10)

In [ ]:

def parse_gzip_medline_str(gzip_str):
    import pubmed_parser as pp
    import unidecode
    
    filepath = gzip_str[0]
    xml_string = gzip_str[1]
    _, file_name = path.split(filepath)
    # decompress gzip

    articles = pp.parse_medline_xml(xml_string)
    return [Row(file_name=file_name, **article_dict)
            for article_dict in articles]



In [ ]:
docs = spark.sparkContext.wholeTextFiles('test/test/pubmed21n0001.xml.gz')
dicts = docs.flatMap(parse_gzip_medline_str)
dicts.take(10)



In [ ]:
!hdfs dfs 

In [ ]:
spark.stop()

In [ ]:
def test_medline_parquet(file_path):
    import subprocess
    (ret, out, err)= run_cmd(['hdfs', 'dfs', '-ls', 'test/test/'])
    
    return (out,err)

In [ ]:
def run_cmd(args_list):
        """
        run linux commands
        """
        # import subprocess
        print('Running system command: {0}'.format(' '.join(args_list)))
        proc = subprocess.Popen(args_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        s_output, s_err = proc.communicate()
        s_return =  proc.returncode
        return s_return, s_output, s_err 

In [ ]:
(ret, out, err)= run_cmd(['hdfs', 'dfs', '-ls', 'test/test/'])
out.decode('ascii').split('\n')

In [ ]:
err = test_p.map(test_medline_parquet).take(1)

In [ ]:
err[0][1].decode('ascii').split('\n')

In [ ]:
tet = spark.sparkContext.wholeTextFiles('test/test/*.xml.gz')

In [ ]:
tet.map(lambda x : x[0]).take(1)

In [ ]:
pa.hdfs.connect()

In [ ]:
spark.stop()